In [1]:
#
#   TfLearn version of DeepMNIST
#

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/Users/jangalask/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jangalask/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jangalask/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jangalask/.local/lib/python3.7/site-packages/tensorflow/python/framework

In [2]:
# Create input object which reads data from MNIST datasets.  Perform one-hot encoding to define the digit
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Reference to TFLearn library
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression


In [4]:
image_rows = 28
image_cols = 28

# reshape the training and test images to 28 X 28 X 1 
train_images = mnist.train.images.reshape(mnist.train.images.shape[0],image_rows, image_cols, 1)
test_images =  mnist.test.images.reshape(mnist.test.images.shape[0], image_rows, image_cols, 1)


In [5]:
num_classes = 10
keep_prob = 0.5                 # fraction to keep (0-1.0)

# Define the shape of the data coming into the NN
input = input_data(shape=[None, 28, 28, 1], name='input')

In [6]:
# Do convolution on images, add bias and push through RELU activation
network = conv_2d(input, nb_filter=32, filter_size=3, activation='relu', regularizer="L2")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [7]:
#   Notice name was not specified.  The name is defaulted to "Conv2D", and will be postfixed with "_n" 
#   where n is the number of the occurance.  Nice!
# take results and run through max_pool
network = max_pool_2d(network, 2)


In [8]:
# 2nd Convolution layer
# Do convolution on images, add bias and push through RELU activation
network = conv_2d(network, nb_filter=64, filter_size=3, activation='relu', regularizer="L2")
# take results and run through max_pool
network = max_pool_2d(network, 2)

In [9]:
# Fully Connected Layer
network = fully_connected(network, 128, activation='tanh')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# dropout some neurons to reduce overfitting
network = dropout(network, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
# Readout layer
network = fully_connected(network, 10, activation='softmax')

# Set loss and measurement, optimizer
network = regression(network, optimizer='adam', learning_rate=0.01,
                        loss='categorical_crossentropy', name='target')


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# Training
num_epoch = 2   # number of times through the data.
model = tflearn.DNN(network, tensorboard_verbose=0)  # for more info in tenorboard turn on tensorboard_verbose
model.fit({'input': train_images}, {'target': mnist.train.labels}, n_epoch=num_epoch,
           validation_set=({'input': test_images}, {'target': mnist.test.labels}),
            show_metric=True, run_id='TFLearn_DeepMNIST')


Training Step: 1719  | total loss: 0.13044 | time: 24.342s
| Adam | epoch: 002 | loss: 0.13044 - acc: 0.9637 -- iter: 54976/55000
Training Step: 1720  | total loss: 0.12966 | time: 25.937s
| Adam | epoch: 002 | loss: 0.12966 - acc: 0.9626 | val_loss: 0.07010 - val_acc: 0.9781 -- iter: 55000/55000
--


In [13]:
model.save('./tflearn_models/tflearn_deep_mnist.tflearn')

INFO:tensorflow:/Users/jangalask/Desktop/Interpretability/tensorflow_learnings/tflearn_models/tflearn_deep_mnist.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
model.load('./tflearn_models/tflearn_deep_mnist.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/jangalask/Desktop/Interpretability/tensorflow_learnings/tflearn_models/tflearn_deep_mnist.tflearn


In [15]:
model.predict(test_images)

array([[1.0634910e-09, 7.4724610e-08, 2.7610191e-08, ..., 9.9999607e-01,
        8.1086764e-09, 3.6721176e-06],
       [1.1055515e-06, 2.4576730e-05, 9.9988735e-01, ..., 1.8131640e-06,
        5.6608827e-05, 6.6976192e-08],
       [5.1079940e-10, 9.9999630e-01, 1.0830498e-07, ..., 5.9902140e-07,
        5.5654688e-08, 2.2482451e-08],
       ...,
       [1.8083367e-10, 1.1059561e-06, 3.4482670e-09, ..., 2.8582750e-07,
        2.8374402e-06, 5.7009470e-06],
       [7.6818509e-07, 5.6558527e-09, 1.8347932e-09, ..., 6.8960211e-09,
        1.4870376e-03, 2.1603993e-04],
       [1.0308732e-05, 6.7612815e-10, 4.4818798e-08, ..., 1.5214066e-10,
        7.6054966e-06, 2.7178078e-07]], dtype=float32)